# 📦 Import Libraries
Import required Python packages and modules.

In [5]:
from typing import *
import networkx as nx
import csv

# 🏗️ Build Graph

Read CSV files and **build graphs**.
use default pass `data/following.csv` to create a graph


In [44]:
def build_graph(path="../data/following.csv"):
    try:
        with open(path, newline='', encoding='utf-8') as f:
            reader = csv.reader(f)
            # skip first line (source_id,relation,target_id)
            next(reader)
            G = nx.MultiDiGraph()
            for src, rel, tgt in reader:
                add_edge(G, src, tgt, rel)
    except Exception as e:
        raise RuntimeError(f"Unable to open file in this path: '{path}'") from e
    # return graph
    return G

default_attrs = {"type": "user", "color": "white", "size": "3.14"}
def add_edge(G, src, tgt, rel):
    if src not in G:
        G.add_node(src, **default_attrs)
    if tgt not in G:
        G.add_node(tgt, **default_attrs)
    G.add_edge(src, tgt, relation=rel)

# 🔢 Node Degree Calculator
Compute **in-degree** and **out-degree** of nodes in directed graphs,
or total degree in undirected graphs.

In [7]:
def degree_calculator(
        G: Union[nx.Graph, nx.DiGraph, nx.MultiGraph, nx.MultiDiGraph],
        node: Optional[Any] = None
) -> Dict[Any, Union[int, Dict[str, int]]]:
    if G is None or G.number_of_nodes() == 0:
        raise ValueError("The input graph is empty or not loaded.")

    if node is not None and not G.has_node(node):
        raise ValueError(f"The node '{node}' does not exist in the graph.")

    results: Dict[Any, Union[int, Dict[str, int]]] = {}
    nodes = [node] if node is not None else G.nodes()

    if nx.is_directed(G):
        for n in nodes:
            results[n] = {"in_degree": G.in_degree(n),"out_degree": G.out_degree(n)}
    else:
        for n in nodes:
            results[n] = G.degree(n)

    return results

# 🔗 Connected Components Finder
Detect **strongly connected components** in directed graphs
and **connected components** in undirected graphs .

In [8]:
def find_connected_components(
        G: Union[nx.Graph, nx.DiGraph, nx.MultiGraph, nx.MultiDiGraph],
) -> List[Set[Union[int, str]]]:
    if G is None or G.number_of_nodes() == 0:
        raise ValueError("The input graph is empty or not loaded.")

    if nx.is_directed(G):
        components = list(nx.strongly_connected_components(G))
    else:
        components = list(nx.connected_components(G))

    return components

# 🛣️ Find the Shortest Path
Use **Dijkstra's algorithm** to compute the **shortest path** and **distance**
between two nodes in agraph.

In [9]:
def find_shortest_path(
        G: Union[nx.Graph, nx.DiGraph, nx.MultiGraph, nx.MultiDiGraph],
        source: Any,
        target: Any,
        weight: Optional[str] = None
) -> Optional[Tuple[List[Any], float]]:
    if G is None or G.number_of_nodes() == 0:
        raise ValueError("The input graph is empty or not loaded.")

    if not G.has_node(source) or not G.has_node(target):
        raise ValueError(f"Source node '{source}' or target node '{target}' not found in the graph.

    try:
        path = nx.dijkstra_path(G, source, target, weight=weight)
        distance = nx.dijkstra_path_length(G, source, target, weight=weight)
        return path, distance
    except nx.NodeNotFound:
        raise ValueError(f"Source node '{source}' or target node '{target}' not found in the graph.")
    except nx.NetworkXNoPath:
        print(f"No path exists between {source} and {target}.")
        return None

# 🌍 Centrality Analysis
Analyze node importance in the network using:
- Degree Centrality
- In/Out Degree (for directed graphs)
- Betweenness
- Closeness
- Eigenvector Centrality

In [10]:
def analyze_centrality(G):
    if G is None or G.number_of_nodes() == 0:
        print("Please load a graph first.")
        return

    print("\n--- Centrality Analysis ---")

    print("\n1. Degree Centrality (higher is more important):")
    degree_centrality = nx.degree_centrality(G)
    for node, centrality in sorted(degree_centrality.items(), key=lambda item: item[1], reverse=True)[:5]:
        print(f"- {node}: {centrality:.3f}")
    if nx.is_directed(G):
        print("\n   → In-Degree Centrality")
        in_degree_centrality = nx.in_degree_centrality(G)
        for node, centrality in sorted(in_degree_centrality.items(), key=lambda item: item[1], reverse=True)[:5]:
            print(f"- {node}: {centrality:.3f}")

        print("\n   → Out-Degree Centrality")
        out_degree_centrality = nx.out_degree_centrality(G)
        for node, centrality in sorted(out_degree_centrality.items(), key=lambda item: item[1], reverse=True)[:5]:
            print(f"- {node}: {centrality:.3f}")

    print("\n2. Betweenness Centrality (nodes on shortest paths):")
    betweenness_centrality = nx.betweenness_centrality(G)
    for node, centrality in sorted(betweenness_centrality.items(), key=lambda item: item[1], reverse=True)[:5]:
        print(f"- {node}: {centrality:.3f}")

    print("\n3. Closeness Centrality (average distance to all others):")
    closeness_centrality = nx.closeness_centrality(G)
    for node, centrality in sorted(closeness_centrality.items(), key=lambda item: item[1], reverse=True)[:5]:
        print(f"- {node}: {centrality:.3f}")

    print("\n4. Eigenvector Centrality (nodes on shortest paths):")